# Get full schedule

```
curl --location --request POST 'https://events.rainfocus.com/api/search' \
--header 'authority: events.rainfocus.com' \
--header 'rfapiprofileid: OpqJ1cGwrxAAMhMxeCpVuSZqCeE8c3uK' \
--header 'rfwidgetid: qzPKy8HO0zOPeC9HX1kwPRe2UN2kSHqd' \
--header 'Content-Type: application/x-www-form-urlencoded' \
--data-urlencode 'type=session' \
--data-urlencode 'size=200' \
--data-urlencode 'browserTimezone=America/Los_Angeles'
```

Save the API response to a file `schedule.json`

In [1]:
import json
from datetime import datetime
from dateutil import tz
import pytz

In [2]:
import json
  
f = open('schedule.json')
data = json.load(f)
f.close()

In [3]:
len(data['sectionList'][0]['items'])

293

In [4]:
# data['sectionList'][0]['items'][0]['times']

In [5]:
local_zone = tz.tzlocal()

def parseTimes(times):
    return {
        'startTime': datetime.strptime(times['utcStartTime'], '%Y/%m/%d %H:%M:%S').replace(tzinfo=pytz.UTC).astimezone(local_zone),
        'endTime': datetime.strptime(times['utcEndTime'], '%Y/%m/%d %H:%M:%S').replace(tzinfo=pytz.UTC).astimezone(local_zone),
        'date': times['date'],
        'room': times['room'],
        'isHidden': times['isHidden'],
        'webinarRequiresEnrollment': times['webinarRequiresEnrollment'],
        'capacity': times['capacity'],
        'inPersonTime': times['inPersonTime']
    }


def parseEvent(event):
    return {
        'code': event['code'],
        'title': event['title'],
        'type': event['type'],
        'abstract': event['abstract'],
        'testRecord': event['testRecord'],
        'times': list(
            filter(lambda x: x['inPersonTime'], 
                   map(parseTimes, 
                       event['times'] if 'times' in event else [])
                  )
            )
    }

In [6]:
schedule = list(map(parseEvent, data['sectionList'][0]['items']))

In [7]:
filteredSchedule = []

for s in schedule:
    for t in s['times']:
        filteredSchedule.append({**s, **t})
        
for s in filteredSchedule:
    del s['times']

In [8]:
# filteredSchedule

In [9]:
def startTime(x):
    return x['startTime']

filteredSchedule.sort(key=startTime)

In [10]:
import pandas
pandas.set_option('display.max_colwidth', 0)

In [11]:
df = pandas.DataFrame(filteredSchedule)


In [12]:
currTime = pandas.Timestamp(datetime.now().astimezone(local_zone))
df[df['startTime'] >= currTime].head(50)

,code,title,type,abstract,testRecord,startTime,endTime,date,room,isHidden,webinarRequiresEnrollment,capacity,inPersonTime
7,PLA1856,Accelerate Derived Value with the Flexibility of Multi-platform Splunk,Theater Session,"Keeping up with the demands of a growing machine data platform can be a challenge. Deloitte DataPaaS simplifies deployments at scale, reducing the TCO of adopting data platforms like Splunk. DataPaaS allows customers and users to focus on outcomes, use cases, and value. Dell Technologies provides infrastructure platform capabilities for Splunk and DataPaaS across the entire value chain including compute, high-performance storage, and cost-optimized storage for Cold, Frozen, and SmartStore. The Deloitte and Dell Technologies alliance helps drive digital transformation for our clients and maximize value, solve problems, and drive efficiencies across the enterprise.\nThis session is first-come, first-served. Adding the session to your agenda does not guarantee a spot.",False,2022-06-14 11:30:00-07:00,2022-06-14 11:50:00-07:00,2022-06-14,Theater in source=*Pavilion,False,False,10000,True
8,BAR1958,Python 3!,Builder Bar,Did you hear that Splunk Enterprise is now Python 3-only? Come and hear how this change impacts your apps in Splunk.,False,2022-06-14 11:30:00-07:00,2022-06-14 11:50:00-07:00,2022-06-14,Builder Bar,False,False,5000,True
9,SEC1304C,A Beginner’s Guide to SOAR: Automate 5 Basic Security Processes in Under 30 Minutes,Breakout Sessions,"Let’s face it, manually operating repetitive security processes every day is boring - creating fatigue and burnout for you and your team. Enriching URLs. IP reputation checks. Blocking domains and file hashes. These are critical tasks that can now be offloaded to machines to automate the basics. Join us to learn how to simply build five Splunk® SOAR playbooks to respond to today’s most common security alerts automatically. Prepare to launch and get ready to SOAR!",False,2022-06-14 12:00:00-07:00,2022-06-14 12:40:00-07:00,2022-06-14,Grand Ballroom - 119,False,False,400,True
10,OBS1299C,AIOps Use Cases With Splunk. No Bullsh… Just IT!,Breakout Sessions,"AIOps is YABW (Yet another buzz word). When I hear people say they are looking for an AIOps solution, often they do not have specific examples in mind, and THAT’S OK! In my experience, the easiest way to demystify a murky topic is by providing meaningful, real-world examples that you can quickly understand and apply!\n\nWe’re taking that murky AIOps term head-on! This session is full of specific examples where you can use out-of-the-box solutions like Splunk AI & ML and other intelligence to increase the IQ of your operational monitoring. Don’t let anyone tell you that their software is an AIOps platform without understanding WHAT specific intelligence it provides. More importantly, HOW you can use that intelligence to up your Observability game.",False,2022-06-14 12:00:00-07:00,2022-06-14 12:40:00-07:00,2022-06-14,Chairman Ballroom - 369,False,False,296,True
11,COM1468A,All in on Climate: Using Data To Drive Climate Solutions and Reduce Emissions in the Race to Net-zero,Panel Discussions,"While the world’s carbon emissions are increasing at an alarming rate, so is the data generated by devices, buildings, transport and other carbon-emitting systems. This data is key to finding actionable solutions to the global climate crisis. Chaired by Splunk’s Director, ESG and Climate, Carrie Christopher, this panel session features contributions from McLaren Racing to talk about the net-zero challenge in F1, BSA, The Software Alliance on global principles for sustainability, and Chief Technical Advisor, EMEA, Mark Woods with a demonstration of the art of the possible.\n",False,2022-06-14 12:00:00-07:00,2022-06-14 12:40:00-07:00,2022-06-14,Grand Ballroom - 118,False,False,286,True
12,FEA1818,A Quantum Life: My Unlikely Journey from the Street to the Stars,Feature Sessions,"<p>Dr. Hakeem Oluseyi grew up in some of the roughest neighborhoo

In [13]:
df.to_csv('sorted.csv')